# FlightAI

FlightAI is a sophisticated virtual assistant for a premium airline, designed to provide customers with accurate and helpful information about flights, bookings, and travel services. The system integrates a conversational AI interface with a mock airline database to simulate real-world interactions between customers and an airline's digital assistant.

## System Architecture

The project is built using the following core components:

1. **Conversational AI Engine**: Utilizes the Ollama API with a Llama 3.2 model to power natural language understanding and generation.
2. **Mock Airline Database**: A Python class that simulates an airline's backend systems with flight information, user profiles, and business logic.
3. **Tool Functions**: A set of defined functions that allow the AI to query the database for specific information.
4. **Gradio Web Interface**: A modern, responsive UI for users to interact with the assistant.

In [1]:
import os
import json
import datetime
import pandas as pd
from openai import OpenAI
import gradio as gr
from typing import Dict, List, Optional, Union, Tuple
import logging
import uuid
import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("FlightAI")

# Initialization
MODEL = "llama3.2"
client = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
logger.info(f"Using model: {MODEL} with Ollama")

def translate_text(text: str, target_language: str) -> str:
    """Translate text to the target language using the AI model"""
    system_prompt = f"""You are a professional translator. 
    Translate the following text to {target_language} exactly as it is, maintaining the same tone and style. 
    Do not add any explanations or additional information. 
    Keep the translation accurate and concise."""
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": text}
            ],
            temperature=0.1,
            max_tokens=500
        )
        translated = response.choices[0].message.content
        return translated.strip()
    except Exception as e:
        logger.error(f"Translation error: {str(e)}")
        return text  # Return original text if translation fails

# System message for the AI assistant
system_message = """
# Role: Premium Airline Support Specialist
**Core Directive:** Combine real-time data verification with expert travel knowledge to deliver precise, actionable responses.

## Operational Protocols
1. **Data Verification Flow:**
   - ALWAYS check tools/database before answering flight status/price questions
   - For bookings: Confirm seat availability FIRST
   - Cross-verify user-provided data (e.g., flight numbers against schedule)

2. **Response Structure:**
   [Fact] → [Source] → [Next Steps]
   Example: 
   "Flight AI215 to London departs at 08:30 (DB Updated: 2min ago). 
   Current status: On Time. Would you like boarding pass assistance?"

3. **Precision Rules:**
   - Include exact times (UTC+local timezone)
   - Show price breakdown: Base fare + Taxes
   - State data freshness ("Updated 3min ago")
   - Cite policy numbers for regulations

4. **Error Handling:**
   - "I'll verify that" → Check tools → Report result
   - If conflict: "Our systems show X. Let me double-check with live operations"

5. **Context Binding:**
   - Maintain conversation thread: 
     "Following up on your Paris flight inquiry earlier..." 
   - Cross-reference user profile:
     "As a Platinum member, you get priority rebooking"

## Current Flight Operations Context
- Baggage Policy: 23kg + 7kg (EC 261/2004)
- Cancellation Window: 24h free (DOT Rule 24H)
- Partner Airlines: Star Alliance Group
- Security Updates: [LIST CURRENT ADVISORIES]

## Prohibited Actions
❌ Never assume flight numbers/routes
❌ No estimations without "approximately" disclaimer
❌ Never quote prices without tax breakdown
❌ Avoid "should be" - use confirmed/expected

## Brand Voice Matrix
| Situation          | Tone               | Formality |
|---------------------|--------------------|-----------|
| Flight disruptions | Empathetic clarity | High      |
| Booking help        | Guided efficiency  | Medium    |
| Loyalty benefits    | Enthusiastic       | Low       |
"""

# Modified chat function to handle translations
def respond(message, original_history, translated_history, current_user_email, language):
    """Process user messages and generate responses in both original and translated languages"""
    if not message.strip():
        return original_history, translated_history, ""
    
    # Generate original response
    bot_message = chat(message, original_history, current_user_email)
    
    # Update original chat history with proper message format
    new_original_history = original_history + [
        {"role": "user", "content": message},
        {"role": "assistant", "content": bot_message}
    ]
    
    # Translate messages
    try:
        user_translation = translate_text(message, language)
        bot_translation = translate_text(bot_message, language)
    except Exception as e:
        logger.error(f"Translation failed: {e}")
        user_translation = message
        bot_translation = bot_message
    
    # Update translated chat history with proper message format
    new_translated_history = translated_history + [
        {"role": "user", "content": user_translation},
        {"role": "assistant", "content": bot_translation}
    ]
    
    return new_original_history, new_translated_history, ""


# Mock database for flight information
class FlightDatabase:
    def __init__(self):
        # Load flight data
        self.flights = {
            "LON123": {"origin": "New York", "destination": "London", "departure": "2025-03-10 08:30", "arrival": "2025-03-10 20:45", "price": 799, "available_seats": 42},
            "PAR456": {"origin": "New York", "destination": "Paris", "departure": "2025-03-15 10:15", "arrival": "2025-03-15 22:30", "price": 899, "available_seats": 28},
            "TKY789": {"origin": "New York", "destination": "Tokyo", "departure": "2025-03-12 23:45", "arrival": "2025-03-14 05:30", "price": 1400, "available_seats": 15},
            "BER101": {"origin": "New York", "destination": "Berlin", "departure": "2025-03-08 14:20", "arrival": "2025-03-09 04:45", "price": 499, "available_seats": 8},
            "ROM202": {"origin": "New York", "destination": "Rome", "departure": "2025-03-20 12:00", "arrival": "2025-03-21 02:15", "price": 849, "available_seats": 22},
            "SYD303": {"origin": "New York", "destination": "Sydney", "departure": "2025-03-18 06:45", "arrival": "2025-03-19 22:30", "price": 1800, "available_seats": 31},
            "HKG404": {"origin": "New York", "destination": "Hong Kong", "departure": "2025-03-25 19:10", "arrival": "2025-03-27 01:45", "price": 1250, "available_seats": 18},
            "DXB505": {"origin": "New York", "destination": "Dubai", "departure": "2025-03-22 23:55", "arrival": "2025-03-24 09:30", "price": 1100, "available_seats": 25},
        }
        
        # Flight status updates (simulating real-time data)
        self.flight_status = {
            "LON123": "On Time",
            "PAR456": "Delayed 30 minutes",
            "TKY789": "Boarding",
            "BER101": "Gate Closed",
            "ROM202": "On Time",
            "SYD303": "Delayed 45 minutes",
            "HKG404": "On Time",
            "DXB505": "Gate Change - Now Gate B12",
        }
        
        # Destinations and their base prices
        self.destinations = {
            "london": {"price": 799, "code": "LHR", "popular_attractions": ["Big Ben", "London Eye", "Buckingham Palace"]},
            "paris": {"price": 899, "code": "CDG", "popular_attractions": ["Eiffel Tower", "Louvre Museum", "Notre-Dame Cathedral"]},
            "tokyo": {"price": 1400, "code": "HND", "popular_attractions": ["Tokyo Skytree", "Senso-ji Temple", "Shibuya Crossing"]},
            "berlin": {"price": 499, "code": "TXL", "popular_attractions": ["Brandenburg Gate", "Berlin Wall", "Reichstag Building"]},
            "rome": {"price": 849, "code": "FCO", "popular_attractions": ["Colosseum", "Vatican City", "Trevi Fountain"]},
            "sydney": {"price": 1800, "code": "SYD", "popular_attractions": ["Sydney Opera House", "Harbour Bridge", "Bondi Beach"]},
            "hong kong": {"price": 1250, "code": "HKG", "popular_attractions": ["Victoria Peak", "Disneyland", "Tian Tan Buddha"]},
            "dubai": {"price": 1100, "code": "DXB", "popular_attractions": ["Burj Khalifa", "Palm Jumeirah", "Dubai Mall"]},
        }
        
        # Loyalty program tiers and benefits
        self.loyalty_tiers = {
            "Silver": {"points_required": 5000, "benefits": ["Priority Check-in", "Extra Baggage Allowance"]},
            "Gold": {"points_required": 25000, "benefits": ["Lounge Access", "Priority Boarding", "Free Seat Selection"]},
            "Platinum": {"points_required": 50000, "benefits": ["Complimentary Upgrades", "Dedicated Support Line", "Partner Lounge Access Worldwide"]}
        }
        
        # Mock user database for testing
        self.users = {
            "john.doe@example.com": {
                "name": "John Doe",
                "loyalty_tier": "Gold",
                "loyalty_points": 28540,
                "upcoming_flights": ["LON123"],
                "preferences": {"seat": "window", "meal": "vegetarian"}
            },
            "jane.smith@example.com": {
                "name": "Jane Smith",
                "loyalty_tier": "Platinum",
                "loyalty_points": 62350,
                "upcoming_flights": ["TKY789", "ROM202"],
                "preferences": {"seat": "aisle", "meal": "regular"}
            }
        }
        
    def get_ticket_price(self, destination_city: str, class_type: str = "economy", date: Optional[str] = None) -> Dict:
        """Get the price of a ticket to the destination city with various parameters"""
        logger.info(f"Getting ticket price for {destination_city}, class: {class_type}, date: {date}")
        city = destination_city.lower()
        
        if city not in self.destinations:
            return {"price": "Unknown", "currency": "USD", "destination": destination_city}
        
        # Base price from the destinations dictionary
        base_price = self.destinations[city]["price"]
        
        # Apply multipliers based on class
        multipliers = {
            "economy": 1.0,
            "premium economy": 1.5,
            "business": 2.5,
            "first": 4.0
        }
        
        # Default to economy if class type not recognized
        class_multiplier = multipliers.get(class_type.lower(), 1.0)
        
        # Apply date-based adjustments (simplified for demonstration)
        date_adjustment = 0
        if date:
            try:
                travel_date = datetime.datetime.strptime(date, "%Y-%m-%d").date()
                today = datetime.datetime.now().date()
                days_until_travel = (travel_date - today).days
                
                if days_until_travel < 7:  # Last minute booking
                    date_adjustment = 0.25  # 25% premium
                elif days_until_travel > 90:  # Far in advance
                    date_adjustment = -0.1  # 10% discount
                    
            except ValueError:
                # Invalid date format, ignore date adjustment
                pass
        
        final_price = base_price * class_multiplier * (1 + date_adjustment)
        
        return {
            "price": f"${int(final_price)}",
            "currency": "USD",
            "destination": destination_city,
            "class": class_type,
            "airport_code": self.destinations[city]["code"]
        }
    
    def search_flights(self, origin: str, destination: str, date: Optional[str] = None) -> List[Dict]:
        """Search for flights matching the given criteria"""
        logger.info(f"Searching flights from {origin} to {destination} on {date}")
        matching_flights = []
        
        dest_lower = destination.lower()
        orig_lower = origin.lower()
        
        for flight_id, details in self.flights.items():
            if details["destination"].lower() == dest_lower and details["origin"].lower() == orig_lower:
                # If date is specified, check if it matches
                if date:
                    flight_date = details["departure"].split()[0]
                    if flight_date != date:
                        continue
                
                flight_info = {
                    "flight_id": flight_id,
                    "origin": details["origin"],
                    "destination": details["destination"],
                    "departure": details["departure"],
                    "arrival": details["arrival"],
                    "price": details["price"],
                    "available_seats": details["available_seats"],
                    "status": self.flight_status.get(flight_id, "Status Unknown")
                }
                matching_flights.append(flight_info)
        
        return matching_flights
    
    def get_flight_status(self, flight_id: str) -> Dict:
        """Get the current status of a flight"""
        logger.info(f"Getting status for flight {flight_id}")
        if flight_id in self.flights and flight_id in self.flight_status:
            return {
                "flight_id": flight_id,
                "status": self.flight_status[flight_id],
                "origin": self.flights[flight_id]["origin"],
                "destination": self.flights[flight_id]["destination"],
                "departure": self.flights[flight_id]["departure"],
                "arrival": self.flights[flight_id]["arrival"]
            }
        return {"flight_id": flight_id, "status": "Flight not found"}
    
    def get_destination_info(self, destination: str) -> Dict:
        """Get information about a destination city"""
        logger.info(f"Getting information for destination {destination}")
        city = destination.lower()
        if city in self.destinations:
            return {
                "destination": destination.title(),
                "airport_code": self.destinations[city]["code"],
                "popular_attractions": self.destinations[city]["popular_attractions"]
            }
        return {"destination": destination, "info": "Destination information not available"}
    
    def get_user_info(self, email: str) -> Dict:
        """Get user profile and loyalty information"""
        logger.info(f"Getting user info for {email}")
        if email in self.users:
            user = self.users[email]
            tier = user["loyalty_tier"]
            return {
                "name": user["name"],
                "loyalty_tier": tier,
                "loyalty_points": user["loyalty_points"],
                "tier_benefits": self.loyalty_tiers[tier]["benefits"],
                "upcoming_flights": [self.get_flight_status(flight_id) for flight_id in user["upcoming_flights"]],
                "preferences": user["preferences"]
            }
        return {"email": email, "info": "User not found"}
    
    def check_baggage_allowance(self, loyalty_tier, travel_class="economy"):
        """Get baggage allowance based on loyalty tier and travel class"""
        logger.info(f"Checking baggage allowance for {loyalty_tier} tier, {travel_class} class")
        # Base allowance by class
        base_allowance = {
            "economy": {"checked": 1, "weight": "23kg", "cabin": "7kg"},
            "premium economy": {"checked": 2, "weight": "23kg", "cabin": "7kg"},
            "business": {"checked": 2, "weight": "32kg", "cabin": "10kg"},
            "first": {"checked": 3, "weight": "32kg", "cabin": "10kg"}
        }
        
        # Tier bonuses
        tier_bonus = {
            "none": {"checked": 0, "weight_bonus": "0kg"},
            "silver": {"checked": 1, "weight_bonus": "0kg"},
            "gold": {"checked": 1, "weight_bonus": "10kg"},
            "platinum": {"checked": 2, "weight_bonus": "10kg"}
        }
        
        travel_class = travel_class.lower() if travel_class else "economy"
        loyalty_tier = loyalty_tier.lower() if loyalty_tier else "none"
        
        if travel_class not in base_allowance:
            travel_class = "economy"  # Default
            
        if loyalty_tier not in tier_bonus:
            loyalty_tier = "none"  # Default
            
        base = base_allowance[travel_class]
        bonus = tier_bonus[loyalty_tier]
    
        return {
            "checked_bags": base["checked"] + bonus["checked"],
            "weight_per_bag": base["weight"],
            "weight_bonus": bonus["weight_bonus"],
            "cabin_baggage": base["cabin"],
            "travel_class": travel_class.title(),
            "loyalty_tier": loyalty_tier.title()
        }

class BookingSystem:
    def __init__(self, flight_database):
        self.flight_db = flight_database
        self.bookings = {}
        
    def create_booking(self, email: str, flight_id: str, passenger_name: str = None, 
                      seat_preference: str = None, meal_preference: str = None) -> dict:
        if flight_id not in self.flight_db.flights:
            return {"error": "Flight not found"}
        
        flight_info = self.flight_db.flights[flight_id]
        if flight_info['available_seats'] <= 0:
            return {"error": "No seats available on this flight"}
        
        booking_ref = str(uuid.uuid4())[:8].upper()
        user_info = self.flight_db.get_user_info(email)
        loyalty_tier = user_info.get('loyalty_tier', 'None')
        
        booking_details = {
            "booking_reference": booking_ref,
            "email": email,
            "flight_id": flight_id,
            "flight_details": {
                "origin": flight_info['origin'],
                "destination": flight_info['destination'],
                "departure": flight_info['departure'],
                "arrival": flight_info['arrival']
            },
            "passenger_name": passenger_name or (user_info.get('name') if 'name' in user_info else 'Unknown'),
            "seat_preference": seat_preference,
            "meal_preference": meal_preference,
            "loyalty_tier": loyalty_tier,
            "booking_date": datetime.datetime.now().isoformat(),
            "status": "Confirmed"
        }
        
        flight_info['available_seats'] -= 1
        self.bookings[booking_ref] = booking_details
        
        return booking_details
    
    def cancel_booking(self, booking_reference: str) -> dict:
        if booking_reference not in self.bookings:
            return {"error": "Booking not found"}
        
        booking = self.bookings[booking_reference]
        flight_id = booking['flight_id']
        
        if flight_id in self.flight_db.flights:
            self.flight_db.flights[flight_id]['available_seats'] += 1
        
        del self.bookings[booking_reference]
        
        return {
            "status": "Cancelled",
            "booking_reference": booking_reference,
            "message": "Booking successfully cancelled"
        }
    
    def get_booking(self, booking_reference: str) -> dict:
        return self.bookings.get(booking_reference, {"error": "Booking not found"})
# Initialize our database
flight_db = FlightDatabase()
flight_db.booking_system = BookingSystem(flight_db)

# Define our tool functions
def create_booking(email, flight_id, passenger_name=None, seat_preference=None, meal_preference=None):
    return flight_db.booking_system.create_booking(
        email, flight_id, passenger_name, seat_preference, meal_preference
    )

def cancel_booking(booking_reference):
    return flight_db.booking_system.cancel_booking(booking_reference)

def get_booking(booking_reference):
    return flight_db.booking_system.get_booking(booking_reference)
    
def get_ticket_price(destination_city, class_type="economy", date=None):
    """Get the price of a ticket to the destination city"""
    return flight_db.get_ticket_price(destination_city, class_type, date)

def search_flights(origin, destination, date=None):
    """Search for flights between origin and destination"""
    return flight_db.search_flights(origin, destination, date)

def get_flight_status(flight_id):
    """Get the current status of a flight"""
    return flight_db.get_flight_status(flight_id)

def get_destination_info(destination):
    """Get information about a destination city"""
    return flight_db.get_destination_info(destination)

def get_user_info(email):
    """Get user profile and loyalty information"""
    return flight_db.get_user_info(email)

def check_baggage_allowance(loyalty_tier, travel_class):
    """Get baggage allowance based on loyalty tier and travel class"""
    return flight_db.check_baggage_allowance(loyalty_tier, travel_class)

# Tool definitions for the AI model
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Get the price of a ticket to the destination city with optional class and date parameters",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {
                        "type": "string",
                        "description": "The city that the customer wants to travel to",
                    },
                    "class_type": {
                        "type": "string",
                        "description": "The class of travel (economy, premium economy, business, first)",
                        "enum": ["economy", "premium economy", "business", "first"]
                    },
                    "date": {
                        "type": "string",
                        "description": "The date of travel in YYYY-MM-DD format",
                    }
                },
                "required": ["destination_city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_flights",
            "description": "Search for flights between origin and destination with optional date",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {
                        "type": "string",
                        "description": "The city of origin",
                    },
                    "destination": {
                        "type": "string",
                        "description": "The destination city",
                    },
                    "date": {
                        "type": "string",
                        "description": "The date of travel in YYYY-MM-DD format",
                    }
                },
                "required": ["origin", "destination"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_flight_status",
            "description": "Get the current status of a flight",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_id": {
                        "type": "string",
                        "description": "The flight ID or number",
                    }
                },
                "required": ["flight_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_destination_info",
            "description": "Get information about a destination city",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination": {
                        "type": "string",
                        "description": "The destination city name",
                    }
                },
                "required": ["destination"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_user_info",
            "description": "Get user profile and loyalty information",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "The email address of the user",
                    }
                },
                "required": ["email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_baggage_allowance",
            "description": "Get baggage allowance based on loyalty tier and travel class",
            "parameters": {
                "type": "object",
                "properties": {
                    "loyalty_tier": {
                        "type": "string",
                        "description": "The loyalty tier of the passenger (None, Silver, Gold, Platinum)",
                        "enum": ["None", "Silver", "Gold", "Platinum"]
                    },
                    "travel_class": {
                        "type": "string",
                        "description": "The class of travel (economy, premium economy, business, first)",
                        "enum": ["economy", "premium economy", "business", "first"]
                    }
                },
                "required": ["loyalty_tier", "travel_class"]
            }
        }
    }
]

booking_tools = [
    {
        "type": "function",
        "function": {
            "name": "create_booking",
            "description": "Create a new flight booking for a user",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string", "description": "User's email address"},
                    "flight_id": {"type": "string", "description": "Flight identifier to book"},
                    "passenger_name": {"type": "string", "description": "Name of the passenger (optional)"},
                    "seat_preference": {"type": "string", "description": "Preferred seat type (window/aisle)", "enum": ["window", "aisle"]},
                    "meal_preference": {"type": "string", "description": "Meal preference", "enum": ["vegetarian", "regular", "kosher", "halal"]}
                },
                "required": ["email", "flight_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "cancel_booking",
            "description": "Cancel an existing flight booking",
            "parameters": {
                "type": "object",
                "properties": {
                    "booking_reference": {"type": "string", "description": "Unique booking reference to cancel"}
                },
                "required": ["booking_reference"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_booking",
            "description": "Retrieve details of an existing booking",
            "parameters": {
                "type": "object",
                "properties": {
                    "booking_reference": {"type": "string", "description": "Unique booking reference to retrieve"}
                },
                "required": ["booking_reference"]
            }
        }
    }
]

tools.extend(booking_tools) 

# Function to handle tool calls
def handle_tool_calls(message) -> Tuple[Dict, Dict]:
    """Process tool calls from the AI and return the results"""
    if not hasattr(message, 'tool_calls') or not message.tool_calls:
        return {"role": "tool", "content": "No tool calls found"}, {}
    
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    logger.info(f"Processing tool call: {function_name} with args {arguments}")
    
    # Match function name to appropriate function
    if function_name == "get_ticket_price":
        destination_city = arguments.get('destination_city')
        class_type = arguments.get('class_type', 'economy')
        date = arguments.get('date')
        result = get_ticket_price(destination_city, class_type, date)
    
    elif function_name == "search_flights":
        origin = arguments.get('origin')
        destination = arguments.get('destination')
        date = arguments.get('date')
        result = search_flights(origin, destination, date)
    
    elif function_name == "get_flight_status":
        flight_id = arguments.get('flight_id')
        result = get_flight_status(flight_id)
    
    elif function_name == "get_destination_info":
        destination = arguments.get('destination')
        result = get_destination_info(destination)
    
    elif function_name == "get_user_info":
        email = arguments.get('email')
        result = get_user_info(email)
    
    elif function_name == "check_baggage_allowance":
        loyalty_tier = arguments.get('loyalty_tier')
        travel_class = arguments.get('travel_class', 'economy')  # Default to 'economy' if not provided
        result = check_baggage_allowance(loyalty_tier, travel_class)
    
    # ====== NEW BOOKING-RELATED TOOL HANDLING ====== #
    elif function_name == "create_booking":
        email = arguments.get('email')
        flight_id = arguments.get('flight_id')
        passenger_name = arguments.get('passenger_name')
        seat_preference = arguments.get('seat_preference')
        meal_preference = arguments.get('meal_preference')
        result = create_booking(email, flight_id, passenger_name, seat_preference, meal_preference)
    
    elif function_name == "cancel_booking":
        booking_reference = arguments.get('booking_reference')
        result = cancel_booking(booking_reference)
    
    elif function_name == "get_booking":
        booking_reference = arguments.get('booking_reference')
        result = get_booking(booking_reference)
    
    # ====== END OF NEW BOOKING-RELATED TOOL HANDLING ====== #
    
    else:
        result = {"error": f"Unknown function: {function_name}"}
    
    response = {
        "role": "tool",
        "content": json.dumps(result),
        "tool_call_id": tool_call.id
    }
    
    return response, result

# Helper function to convert older chat format to messages format
def convert_chat_history(history):
    """Convert from older chat format to messages format"""
    messages = []
    for exchange in history:
        if exchange[0] is not None:  # User message
            messages.append({"role": "user", "content": exchange[0]})
        if exchange[1] is not None:  # Assistant message
            messages.append({"role": "assistant", "content": exchange[1]})
    return messages

# Main chat function
def chat(message, history, current_user_email=None):
    """Process user messages and generate AI responses with tool support"""
    # Create messages for the API call
    messages = [{"role": "system", "content": system_message}]
    
    # Convert history from Gradio's format to OpenAI format if needed
    if history and isinstance(history[0], list):
        # Convert old format to messages format
        for user_msg, assistant_msg in history:
            if user_msg is not None:
                messages.append({"role": "user", "content": user_msg})
            if assistant_msg is not None:
                messages.append({"role": "assistant", "content": assistant_msg})
    else:
        # History is already in messages format
        messages.extend(history)
    
    # Add the current message
    messages.append({"role": "user", "content": message})
    
    # Add user context if available
    if current_user_email:
        user_info = get_user_info(current_user_email)
        if "name" in user_info:  # User found
            context = f"The current user is {user_info['name']}, a {user_info['loyalty_tier']} tier member with {user_info['loyalty_points']} loyalty points."
            messages[0]["content"] += f"\n\nCurrent user context: {context}"
    
    # Get response from the AI
    try:
        logger.info(f"Sending request to {MODEL}")
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )
        
        # Handle any tool calls
        if hasattr(response.choices[0], 'finish_reason') and response.choices[0].finish_reason == "tool_calls":
            message_obj = response.choices[0].message
            tool_response, tool_result = handle_tool_calls(message_obj)
            
            # Add the tool call and response to the message history
            messages.append({
                "role": "assistant",
                "content": message_obj.content,
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": tc.type,
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        }
                    } for tc in message_obj.tool_calls
                ]
            })
            messages.append(tool_response)
            
            # Get final response from the AI
            logger.info("Getting final response after tool call")
            response = client.chat.completions.create(
                model=MODEL,
                messages=messages
            )
        
        # Ensure the response is in the correct format
        if hasattr(response.choices[0], 'message'):
            ai_message = response.choices[0].message.content
            return ai_message
        else:
            return "I apologize, but I'm having trouble processing your request at the moment. Please try again later."
        
    except Exception as e:
        logger.error(f"Error in AI request: {str(e)}")
        return f"I apologize, but I'm having trouble processing your request at the moment. Please try again later. (Error: {str(e)})"

# UI Components
custom_theme = gr.themes.Base(
    primary_hue="blue",
    secondary_hue="teal",
    neutral_hue="gray",
    font=[gr.themes.GoogleFont('Poppins'), 'sans-serif']
)

# Apply theme properties using individual settings instead of .set()
custom_css = """
:root {
    --background-fill-primary: #1a1a1a;
    --background-fill-secondary: #2d2d2d;
    --text-color: #ffffff;
    --accent-blue: #38BDF8;
    --accent-teal: #2DD4BF;
}

.gradio-container {
    background: var(--background-fill-primary) !important;
    font-family: 'Poppins', sans-serif;
    color: var(--text-color) !important;
}

@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;500;600&display=swap');

.chatbot {
    background: var(--background-fill-secondary) !important;
    backdrop-filter: blur(10px);
    border-radius: 16px;
    border: 1px solid #3d3d3d !important;
    box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.2);
}

.chatbot .message {
    padding: 16px 20px;
}

.chatbot .user {
    border-left: 4px solid var(--accent-blue);
    background: rgba(56, 189, 248, 0.05) !important;
}

.chatbot .assistant {
    border-left: 4px solid var(--accent-teal);
    background: rgba(45, 212, 191, 0.05) !important;
}

.dataframe {
    border-radius: 12px !important;
    overflow: hidden;
    background: var(--background-fill-secondary) !important;
}

.dataframe table {
    border-radius: 12px;
    color: var(--text-color) !important;
}

button.primary {
    background-color: var(--accent-blue) !important;
    color: #ffffff !important;
    border-color: var(--accent-blue) !important;
}

button.primary:hover {
    background-color: #1e6f9f !important;
}

nput, textarea {
    background-color: var(--background-fill-secondary) !important;
    border-color: #4a4a4a !important;
    color: var(--text-color) !important;
}

.block {
    background-color: var(--background-fill-primary) !important;
    border-color: #4a4a4a !important;
}

.block label {
    color: var(--accent-blue) !important;
}

.block .title {
    color: var(--accent-teal) !important;
}

body {
    color: var(--text-color) !important;
}

a {
    color: var(--accent-blue) !important;
}

.checkbox {
    background-color: var(--background-fill-secondary) !important;
    border-color: #4a4a4a !important;
}

.checkbox-label {
    background-color: var(--background-fill-primary) !important;
}

.chatbot-bubble {
    background-color: var(--background-fill-secondary) !important;
    color: var(--text-color) !important;
}

.chatbot-bubble-user {
    background-color: #2d2d2d !important;
    color: var(--text-color) !important;
}

.accordion {
    border-color: #4a4a4a !important;
}
"""

with gr.Blocks(
    theme=custom_theme,
    css=custom_css
) as demo:
    gr.Markdown(
        """
        <div style="text-align: center; padding: 24px 0;">
            <h1 style="color: var(--accent-blue); margin: 0; font-weight: 600;">✈️ SkyAssistant AI</h1>
            <p style="color: #94a3b8; margin: 8px 0 0; font-size: 0.95em;">
                Your Intelligent Air Travel Companion
            </p>
        </div>
        """
    )
    
    current_user = gr.State("")
    
    with gr.Row(equal_height=True):
        # Left Column: Original Chat
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                height=640,
                show_label=False,
                avatar_images=(
                    "https://api.dicebear.com/7.x/bottts/svg?seed=user&backgroundColor=1e3a8a",
                    "https://api.dicebear.com/7.x/bottts/svg?seed=cloud&backgroundColor=38bdf8"
                ),
                bubble_full_width=False,
                type='messages',
                elem_classes="chatbot"
            )
            
            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Ask about flights, bookings, or travel information...",
                    container=False,
                    scale=9,
                    show_label=False,
                    lines=1,
                    max_lines=3,
                    autofocus=True
                )
                submit_btn = gr.Button("Send", variant="primary", scale=1, min_width=120)
            
            with gr.Row():
                clear_btn = gr.Button("Clear Chat", variant="secondary", size='lg', min_width=120)
                example_selector = gr.Dropdown(
                    choices=[
                        "How much is a ticket to London?",
                        "What's the status of flight LON123?",
                        "Tell me about Tokyo as a destination.",
                        "What's my baggage allowance for business class?",
                        "Find flights from New York to Paris next week",
                        "What are Platinum tier benefits?"
                    ],
                    label="Quick Questions",
                    interactive=True,
                    allow_custom_value=True,
                    container=False,
                    scale=6
                )

        # Right Column: Translation and Tools
        with gr.Column(scale=3):
            # Language Selection
            language = gr.Dropdown(
                choices=["Spanish", "French", "German", "Italian", "Chinese", "Japanese", "Korean", "Russian"],
                value="Spanish",
                label="Translate to",
                interactive=True
            )
            
            # Translated Chat
            translated_chatbot = gr.Chatbot(
                height=640,
                show_label=False,
                avatar_images=(
                    "https://api.dicebear.com/7.x/bottts/svg?seed=user&backgroundColor=1e3a8a",
                    "https://api.dicebear.com/7.x/bottts/svg?seed=cloud&backgroundColor=38bdf8"
                ),
                bubble_full_width=False,
                type='messages',
                elem_classes="chatbot",
                value=[]  # Initialize with an empty list
            )

            # User Profile Section
            with gr.Accordion("🛂 Traveler Profile", open=False):
                with gr.Group():
                    user_email = gr.Textbox(
                        placeholder="Enter email for personalized service",
                        label="Traveler Email",
                        max_lines=1,
                        container=False
                    )
                    with gr.Row():
                        load_profile_btn = gr.Button("Load Profile", variant="secondary", size='sm')
                        clear_profile_btn = gr.Button("Clear", variant="stop", size='sm')
                    user_info_display = gr.JSON(label="Profile Info", visible=False)
            
            # Flight Search Tools
            with gr.Accordion("🔍 Flight Search", open=False):
                with gr.Group():
                    origin_input = gr.Textbox(placeholder="e.g., New York", label="Departure City")
                    destination_input = gr.Textbox(placeholder="e.g., London", label="Destination City")
                    date_input = gr.Textbox(placeholder="YYYY-MM-DD", label="Travel Date")
                    find_flights_btn = gr.Button("Search Flights", variant="primary")
                    flight_results = gr.DataFrame(
                        headers=["Flight", "Departure", "Arrival", "Price", "Status"],
                        datatype=["str", "str", "str", "str", "str"],
                        row_count=8,
                        interactive=True,
                        label="Available Flights"
                    )

            # Quick Links
            with gr.Accordion("ℹ️ Quick Links", open=False):
                gr.Markdown("""
                <div style="padding: 12px 0;">
                    <a href="#" style="color: var(--accent-blue) !important; text-decoration: none; display: block; padding: 8px 0;">📡 Live Flight Status</a>
                    <a href="#" style="color: var(--accent-blue) !important; text-decoration: none; display: block; padding: 8px 0;">🧳 Baggage Policy</a>
                    <a href="#" style="color: var(--accent-blue) !important; text-decoration: none; display: block; padding: 8px 0;">🌟 Loyalty Program</a>
                    <a href="#" style="color: var(--accent-blue) !important; text-decoration: none; display: block; padding: 8px 0;">🌍 Destination Guide</a>
                </div>
                """)

    # Unified Event Handler
    def respond(message, original_history, translated_history, current_user_email, language):
        """Process messages with translation support"""
        if not message.strip():
            return original_history, translated_history, ""
        
        # Get original response
        bot_message = chat(message, original_history, current_user_email)
        
        # Update original chat
        new_original = original_history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": bot_message}
        ]
        
        # Translate content
        try:
            user_tr = translate_text(message, language)
            bot_tr = translate_text(bot_message, language)
        except Exception as e:
            logger.error(f"Translation error: {e}")
            user_tr, bot_tr = message, bot_message
        
        # Update translated chat
        new_translated = translated_history + [
            {"role": "user", "content": user_tr},
            {"role": "assistant", "content": bot_tr}
        ]
        
        return new_original, new_translated, ""

    # Event Connections
    submit_btn.click(
        respond,
        [msg, chatbot, translated_chatbot, current_user, language],
        [chatbot, translated_chatbot, msg],
        queue=False
    ).then(lambda: "", None, [msg])
    
    msg.submit(
        respond,
        [msg, chatbot, translated_chatbot, current_user, language],
        [chatbot, translated_chatbot, msg],
        queue=False
    )
    
    clear_btn.click(
        lambda: ([], [], ""), 
        None, 
        [chatbot, translated_chatbot, msg]
    )
    
    # Profile Handlers
    load_profile_btn.click(
        lambda email: (email, flight_db.get_user_info(email)),
        [user_email],
        [current_user, user_info_display]
    ).then(
        lambda x: gr.update(visible=bool(x)),
        [user_info_display],
        [user_info_display]
    )
    
    clear_profile_btn.click(
        lambda: ("", None),
        None,
        [current_user, user_info_display]
    ).then(
        lambda: gr.update(visible=False),
        None,
        [user_info_display]
    )
    
    # Flight Search Handler
    find_flights_btn.click(
        lambda o,d,t: pd.DataFrame([
            [f["flight_id"], f["departure"], f["arrival"], f"${f['price']}", f["status"]]
            for f in flight_db.search_flights(o,d,t)
        ]),
        [origin_input, destination_input, date_input],
        [flight_results]
    )
    
    example_selector.change(
        lambda ex: ex or "",
        [example_selector],
        [msg]
    )
    
if __name__ == "__main__":
    demo.launch(share=False)

2025-03-08 17:38:04,004 - FlightAI - INFO - Using model: llama3.2 with Ollama
2025-03-08 17:38:05,488 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-03-08 17:38:07,096 - httpx - INFO - HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-03-08 17:38:07,156 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
